# Bow tracking dataset analysis

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import os
import math

## 1. Load dataset from CSV files using Pandas
### Plot basic distribution of dataset by position and HSD

In [ ]:
# Load dataset
dataset_path_1 = "./dataset_collection/dataset/dataset_2024-04-18_18-30-01_formatted.csv"
dataset_path_2 = "./dataset_collection/dataset/dataset_2024-04-22_15-37-44_formatted.csv"
dataset_path_3 = "./dataset_collection/dataset/dataset_2024-04-18_18-30-01_2024-04-22_15-37-44_combined_formatted_avg_samples.csv"
dataset_1 = pd.read_csv(dataset_path_1)
dataset_2 = pd.read_csv(dataset_path_2)
dataset_3 = pd.read_csv(dataset_path_3)
dataset = pd.concat([dataset_1, dataset_2, dataset_3])

# dataset = pd.read_csv("./dataset_collection/dataset/dataset_2024-04-18_18-30-01_2024-04-22_15-37-44_combined_formatted_avg_samples.csv")
# dataset.dropna(inplace=True)
dataset.reset_index(inplace=True)

dataset = dataset[["norm1", "norm2", "norm3", "norm4", "position", "hair_stick_distance"]]

# Load expected position and HSD combinations
dataset_combos_path = "./dataset_collection/dataset_values_to_measure/dataset_combinations.txt"
with open(dataset_combos_path, "r+") as f:
    combos = []
    combos_data = f.readlines()
    for line in combos_data:
        vals = line.split(" ")
        combo = [int(vals[1]), int(vals[2].split(";")[0])]
        combos.append(combo)

# Plot examples per position
position_value_counts = dataset["position"].value_counts()
plt.figure(figsize=(10,2))
plt.bar(position_value_counts.index, position_value_counts, width=18, color="purple")
plt.xlabel("Position [mm]")
plt.ylabel("# of examples")
plt.xlim(-20, 680)
plt.yticks(np.arange(0, 8001, 1000))
plt.show()

# Plot examples per HSD
hsd_value_counts = dataset["hair_stick_distance"].value_counts()
plt.figure(figsize=(6,2))
plt.barh(hsd_value_counts.index, hsd_value_counts, height=1.8, color="darkgreen")
plt.xlabel("# of examples")
plt.ylabel("HSD [mm]")
plt.yticks(np.arange(0, 14, 2))
plt.show()

dataset

## 2. Compute statistics per each sensor for each combination of pos & HSD in the dataset

Per sensor per combination:
- Mean sensor reading
- Median sensor reading
- Standard deviation of sensor reading
- Maximum sensor reading
- Minimum sensor reading

In [ ]:
# General stats per combo - count, means, mins, maxes
combo_examples_df = pd.DataFrame(columns=["position", "hair_stick_distance", "count"])

norm_cols = ["norm1", "norm2", "norm3", "norm4"]

for combo in combos:
    pos_subset = dataset[dataset["position"] == combo[0]]
    pos_hsd_subset = pos_subset[pos_subset["hair_stick_distance"] == combo[1]]
    if len(pos_hsd_subset.index) > 0: # Remove combos with no data
        idx = len(combo_examples_df.index)
        combo_examples_df.loc[idx, "position"] = combo[0]
        combo_examples_df.loc[idx, "hair_stick_distance"] = combo[1]
        combo_examples_df.loc[idx, "count"] = len(pos_hsd_subset[pos_hsd_subset["hair_stick_distance"] == combo[1]].index)
        for col in norm_cols:
            combo_examples_df.loc[idx, f"{col}_mean"] = pos_hsd_subset[col].mean()
            combo_examples_df.loc[idx, f"{col}_median"] = pos_hsd_subset[col].median()
            combo_examples_df.loc[idx, f"{col}_min"] = pos_hsd_subset[col].min()
            combo_examples_df.loc[idx, f"{col}_max"] = pos_hsd_subset[col].max()
            combo_examples_df.loc[idx, f"{col}_std"] = pos_hsd_subset[col].std()

for column in combo_examples_df.columns:
    if combo_examples_df[column].dtype == 'object':
        combo_examples_df[column] = combo_examples_df[column].astype(np.float32)

max_pos = int(combo_examples_df["position"].max())

combo_examples_df

## 3. Visualise the number of dataset examples per combination

In [ ]:
# Plot stacked bars for each displacement, x=position, y=count
colours = ["firebrick", "mediumblue", "darkorange", "green", "crimson", "gold", "purple"]
valid_combos = 0

plt.figure(figsize=(10,4))
for i, pos in enumerate(combo_examples_df["position"].unique()):
    position_subset = combo_examples_df[combo_examples_df["position"] == pos].reset_index(drop=True)
    bottom = 0
    for j, hsd in enumerate(position_subset["hair_stick_distance"].unique()):
        count = position_subset.loc[j, "count"]
        plt.bar(pos, count, bottom=bottom, width=12, color=colours[j], label=int(hsd) if pos == 300 else "", zorder=3, ec="black", lw=0.5)
        bottom += count
        valid_combos += 1

plt.xlabel(f"Bow position [mm] (0mm at tip, {max_pos}mm at frog)")
plt.ylabel("# of examples in dataset")
plt.xlim(-20, 680)
plt.yticks(np.arange(0, 7500, 500))
plt.xticks(np.arange(0, max_pos+1, 20), rotation=90)
plt.legend(title="HSD", fancybox=True, reverse=True)
plt.grid(which="both", color="lightgrey", alpha=0.5, zorder=1)
plt.title("Number of dataset examples per position & HSD combination")
# plt.savefig("./figures/dataset_examples_per_combo.png", dpi=600, bbox_inches="tight")
plt.show()

print(f"Valid combos: {valid_combos}")

## 4. Plot combination statistics from 2. - mean, std. dev., min, max, per sensor
### One subplot per HSD

In [ ]:
# Plot per HSD of values at each position
hsd_uniques = np.sort(dataset["hair_stick_distance"].unique())
cols_to_plot = ["norm1", "norm2", "norm3", "norm4"]
sensor_friendly_names = ["1 - frog/lowest", "2 - lower middle", "3 - upper middle", "4 - tip/highest"]
colours = ["red", "purple", "green", "orange"]
sensor_positions = [562, 465, 189, 69]

ylims = (0, 1)
xlims = (0, max_pos)

fig, axs = plt.subplots(len(hsd_uniques), 1, figsize=(8,12), sharex=True)

for i, hsd in enumerate(hsd_uniques):
    ax = axs[i]
    ax.set_ylim(ylims)
    ax.set_xlim(xlims)
    ax.set_title(f"HSD: {int(hsd)} mm", x=1.03, y=0, rotation="vertical")
    
    hsd_subset = combo_examples_df[combo_examples_df["hair_stick_distance"] == hsd]
    
    for j, sensor in enumerate(cols_to_plot):
        ax.axvline(
            sensor_positions[j],
            ls=":", 
            c=colours[j], 
            lw=1.5, 
            label=f"{sensor_friendly_names[j]}: sensor pos." if i == 0 else ""
        )

        # Plot standard deviation
        ax.plot(
            hsd_subset["position"],
            hsd_subset[f"{sensor}_mean"] - hsd_subset[f"{sensor}_std"],
            color=colours[j],
            linewidth=0.5,
            ls=":",
            label=f"{sensor_friendly_names[j]}: std. dev." if i == 0 else ""
        )

        ax.plot(
            hsd_subset["position"],
            hsd_subset[f"{sensor}_mean"] + hsd_subset[f"{sensor}_std"],
            color=colours[j],
            linewidth=0.5,
            ls=":"
        )
        
        # Plot averages
        ax.plot(
            hsd_subset["position"], 
            hsd_subset[f"{sensor}_mean"], 
            color=colours[j], 
            linewidth=0.75, 
            label=f"{sensor_friendly_names[j]}: mean" if i == 0 else ""
        )

        # Plot mins and maxes
        ax.fill_between(
            hsd_subset["position"], 
            hsd_subset[f"{sensor}_min"], 
            hsd_subset[f"{sensor}_max"],
            color=colours[j],
            linewidth=0,
            alpha=0.075,
            label=f"{sensor_friendly_names[j]}: range" if i == 0 else ""
        )

        # Plot bounds
        left_bound = patches.Rectangle(
            (0,0), 
            hsd_subset["position"].min(), 
            ylims[1], 
            color="lightgray", 
            linewidth=1, 
            alpha=0.1,
            hatch="/"
        )
        
        right_bound = patches.Rectangle(
            (hsd_subset["position"].max(),0), 
            max_pos-hsd_subset["position"].max(), 
            ylims[1], color="lightgray", 
            linewidth=1, 
            alpha=0.1,
            hatch="/"
        )
        
        ax.add_patch(left_bound)
        ax.add_patch(right_bound)

        # Formatting
        ax.set_xlabel("Bow position [mm]" if i == 0 else "")
        ax.xaxis.set_label_position("top")
        ax.set_ylabel("Sensor value")
        ax.set_axisbelow(True)
        ax.grid(visible=True, which="both", color="lightgrey")
        if i == 0:
            ax.set_xticks(np.arange(0,670,20))
            ax.tick_params(top=True, axis="x", which="both", length=3, labeltop=True, rotation=90)
        elif i == len(axs) - 1:
            # ax.set_xticks(sensor_positions)
            ax.tick_params(top=True, bottom=False, axis="x", which="both", length=3, labeltop=False, labelbottom=False)
        else:
            ax.tick_params(top=True, axis="x", which="both", length=3, labeltop=False, labelbottom=False)
        
fig.tight_layout()
fig.suptitle("Bow Tracking Dataset | Sensor Mean / Range / Std. Dev. by Pos & HSD", fontsize=14, y=1.03)
fig.legend(ncols=4, loc="upper center", bbox_to_anchor=[0.5, 0], fontsize=8)
plt.savefig("./dataset_analysis_plots/bt_norm_means_&_ranges_by_hsd.png", dpi=600, transparent=False, bbox_inches="tight")
plt.show()

## 5.a Bin the dataset by sensor readings

In [ ]:
cols = ["norm1", "norm2", "norm3", "norm4"]

n_vertical_bins = 250

quant_dataset = dataset.copy()
quant_dataset.reset_index(inplace=True)

for i, row in quant_dataset.iterrows():
    for col in cols:
        quant_sensor_val = round(row[col] * n_vertical_bins) / n_vertical_bins
        quant_dataset.loc[i, f"{col}_quant"] = quant_sensor_val
    if i % 10000 == 0 and i > 0:
        print(i)

print("DONE")

quant_dataset = quant_dataset[["position", "hair_stick_distance", "norm1_quant", "norm2_quant", "norm3_quant", "norm4_quant"]]
quant_dataset

## 5.b Plot binned data as heatmap
### Shows the spread of values around the mean for each combination

In [ ]:
sensor_cols = ["norm1", "norm2", "norm3", "norm4"]

sensor_colormaps = ["Blues", "Greens", "Purples", "Reds"]

sensor_friendly_names = ["1 (frog)", "2 (lower middle)", "3 (upper middle)", "4 (tip)"]

sensor_positions = [562, 465, 189, 69]

tick_len = 4

ylims = (0,1)
xlims = (0, max_pos)

step = 1 / n_vertical_bins
quant_value_options = np.arange(0, 1.0001, step)
n_quant_value_options = len(quant_value_options)
# print(quant_value_options, n_quant_value_options)

positions = np.sort(quant_dataset["position"].unique())
hsds = np.sort(quant_dataset["hair_stick_distance"].unique())

fig, axs = plt.subplots(len(hsds), 4, figsize=(12,16), sharex=True, sharey=True)

# Iterate through HSD values
for i, hsd in enumerate(hsds):
    hsd_subset = quant_dataset[quant_dataset["hair_stick_distance"] == hsd]

    # Iterate through sensors for current HSD
    for j, sensor in enumerate(sensor_cols):
        
        hsd_sensor_grid = np.zeros((len(quant_value_options), len(positions)))
        # print(hsd_sensor_grid.shape)

        # Iterate through positions for current sensor at current HSD value
        for k, pos in enumerate(positions):
            hsd_pos_subset = hsd_subset[hsd_subset["position"] == pos]
            # print(hsd_pos_subset.shape)

            # Bypass any positions where there is no data for the current sensor and HSD
            if len(hsd_pos_subset.index) > 0:
                n_hsd_pos_examples = len(hsd_pos_subset.index)

                # Iterate through all the possible quantised values
                # To check how many of each are present in the HSD pos sensor combination
                for l, y_idx in enumerate(quant_value_options):
                    
                    hsd_pos_y_idx_count = len(hsd_pos_subset[hsd_pos_subset[f"{sensor}_quant"] == y_idx].index)
                    if hsd_pos_y_idx_count > 0:
                        val_ratio = math.sqrt(hsd_pos_y_idx_count / n_hsd_pos_examples)
                    else:
                        val_ratio = 0
  
                    hsd_sensor_grid[l, k] = val_ratio
        
        # Plot grid to subplot
        ax = axs[i, j]
        heatmap = ax.imshow(hsd_sensor_grid, interpolation="none", cmap=sensor_colormaps[j], aspect="auto", extent=(0, 660, 0, 1), origin="lower", zorder=1)

        if i == 0:
            plt.colorbar(heatmap, ax=axs[:,j], location="top")
        
        combo_hsd_subset = combo_examples_df[combo_examples_df["hair_stick_distance"] == hsd]

        # Plot mean
        ax.plot(combo_hsd_subset["position"], combo_hsd_subset[f"{sensor}_mean"], ls="--", lw=0.75, c="black", zorder=3, label="Sensor value mean" if i == 0 and j == 0 else "")
        ax.grid(which="both", c="lightgrey", alpha=1, zorder=2)

        # Plot bounds
        left_bound = patches.Rectangle(
            (0,0), 
            combo_hsd_subset["position"].min(), 
            ylims[1], 
            color="lightgrey", 
            linewidth=1, 
            alpha=0.5,
            hatch="/"
        )
        
        right_bound = patches.Rectangle(
            (hsd_subset["position"].max(),0), 
            max_pos-combo_hsd_subset["position"].max(), 
            ylims[1],
            color="lightgrey", 
            linewidth=1, 
            alpha=0.5,
            hatch="/",
        )

        ax.add_patch(left_bound)
        ax.add_patch(right_bound)

        for l, sensor_pos in enumerate(sensor_positions):
            ax.axvline(
                sensor_pos, 
                ls=":", 
                c="black", 
                lw=1 
                # label="Sensor positions" if k == 0 else ""
            )

        # Formatting
        ax.set_ylim(*ylims)
        ax.set_xlim(*xlims)

        ax.set_xlabel("Bow position [mm]" if i == len(hsds)-1 else "")
        ax.set_ylabel("Sensor value" if j == 0 else "")

        if i == 0:
            ax.tick_params(top=True, axis="x", which="both", length=tick_len, labeltop=False, direction="in")
        elif i == len(axs) - 1:
            ax.tick_params(top=True, bottom=True, axis="x", which="both", length=tick_len, labeltop=False, labelbottom=True, direction="in")
        else:
            ax.tick_params(top=True, axis="x", which="both", length=tick_len, labeltop=False, labelbottom=False, direction="in")

        if j == 0:
            ax.tick_params(left=True, right=True, axis="y", which="both", length=tick_len, direction="in")
        elif j != len(sensor_cols)-1:
            ax.tick_params(left=True, right=True, axis="y", which="both", length=tick_len, direction="in")
        else:
            ax.tick_params(left=True, right=False, axis="y", which="both", length=tick_len, direction="in")
        
        ax.set_title(f"Sensor {sensor_friendly_names[j]}" if i == 0 else "")

fig.legend(loc="lower center", bbox_to_anchor=(0.5, 0.05))
plt.show()